# RoPE (旋转位置编码)

这篇笔记研究了不同位置编码方法（特别是RoPE及其变体）的衰减特性，主要目的是：

1. 分析位置编码如何影响词向量之间的距离关系
2. 比较不同位置编码方法（绝对位置编码、RoPE、PI-RoPE和NTK-RoPE）在短长上下文中的表现
3. 探索位置编码的平滑衰减特性对模型理解位置关系的影响

**位置编码的作用**
- 使模型能够区分不同位置的词向量
- 保持词之间的距离关系（近的词相关性高，远的词相关性低）

**好的位置编码特性**
1. **平滑衰减**：距离增加时相关性应平滑下降，而非突变
2. **长距离区分能力**：能区分1-10000和1-10000000这样不同量级的距离差异

**主要位置编码方法**
1. **绝对位置编码**（Transformer使用）
2. **旋转位置编码RoPE**（Llama使用）
3. **PI-RoPE**（位置插值）
4. **NTK-RoPE**（基于NTK理论的方法, ）

NTK: 神经切线核（Neural Tangent Kernel，简称NTK）是一种在深度学习领域中被广泛研究的概念，它提供了一种框架来分析和理解神经网络训练过程中的动态行为。NTK是在无限宽度极限下的神经网络中定义的，即当网络的层宽度趋向于无限大时，网络的行为可以通过一个固定的核函数来描述。

In [6]:
from typing import Tuple, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 设置随机种子保证可重复性
torch.manual_seed(42)

In [7]:
# 1. 绝对位置编码 (Transformer原版)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)  # [max_len, d_model]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """输入x形状: [seq_len, batch_size, d_model]"""
        return x + self.pe[:x.size(0)]

# 2. 旋转位置编码 (RoPE)
class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, max_position_embeddings: int = 2048, base: float = 10000.0):
        super().__init__()
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        self._set_cos_sin_cache()

    def _set_cos_sin_cache(self):
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2).float() / self.dim))
        t = torch.arange(self.max_position_embeddings).float()
        
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos(), persistent=False)
        self.register_buffer("sin_cached", emb.sin(), persistent=False)

    def forward(self, x: torch.Tensor, seq_len: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.cos_cached[:seq_len], self.sin_cached[:seq_len]

# 旋转辅助函数
def rotate_half(x: torch.Tensor) -> torch.Tensor:
    """旋转一半的隐藏维度"""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(
    q: torch.Tensor, 
    k: torch.Tensor, 
    cos: torch.Tensor, 
    sin: torch.Tensor
) -> Tuple[torch.Tensor, torch.Tensor]:
    """应用旋转位置编码到查询和键上"""
    cos = cos.unsqueeze(0).unsqueeze(0)  # [1, 1, seq_len, dim]
    sin = sin.unsqueeze(0).unsqueeze(0)  # [1, 1, seq_len, dim]
    
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed

# 计算衰减分数
def compute_decay_scores(encoding_type: str, encoder: nn.Module, dim: int, seq_len: int) -> torch.Tensor:
    """计算位置编码的衰减分数"""
    # 创建测试输入 (模拟注意力查询和键)
    q = torch.ones(1, 1, seq_len, dim)
    k = torch.ones(1, 1, seq_len, dim)
    
    if encoding_type == "absolute":
        # 绝对位置编码直接加到输入上
        pe = encoder.pe[:seq_len].unsqueeze(0).unsqueeze(0)  # [1, 1, seq_len, dim]
        q_embed = q + pe
        k_embed = k + pe
    elif encoding_type == "rope":
        # RoPE需要特殊处理
        cos, sin = encoder(q, seq_len)
        q_embed, k_embed = apply_rotary_pos_emb(q, k, cos, sin)
    
    # 计算第一个位置与其他位置的点积
    scores = q_embed[0, 0, 0] @ k_embed[0, 0].T
    return scores

# 可视化对比
def plot_comparison(pos_scores: torch.Tensor, rope_scores: torch.Tensor, max_len: int = 2048):
    plt.figure(figsize=(12, 6))
    plt.plot(pos_scores[:max_len], label="Absolute Positional Encoding", color="blue")
    plt.plot(rope_scores[:max_len], label="Rotary Positional Encoding (RoPE)", color="red")
    
    plt.title("Positional Encoding Comparison (First {} Positions)".format(max_len))
    plt.xlabel("Relative Position")
    plt.ylabel("Dot Product Score")
    plt.legend()
    plt.grid(True)
    plt.show()

In [8]:
device = "cpu"
dim = 256
seq_len = 4096  # 测试序列长度

# 初始化编码器
pos_encoder = PositionalEncoding(dim, seq_len).to(device)
rope_encoder = RotaryEmbedding(dim, seq_len).to(device)

# 计算衰减分数
pos_scores = compute_decay_scores("absolute", pos_encoder, dim, seq_len)
rope_scores = compute_decay_scores("rope", rope_encoder, dim, seq_len)

# 绘制对比图
plot_comparison(pos_scores, rope_scores, max_len=1024)  # 显示前1024个位置的对比
plot_comparison(pos_scores, rope_scores, max_len=seq_len)  # 显示全部位置的对比


NameError: name 'np' is not defined